# Scikit-learn

![alt](http://www.scatter.com/images/DataLab_logo.jpg)

General documentation and examples for Scikit-learn are available at
- http://scikit-learn.org/stable/

__Relevant Libraries:__
- `Numpy` Adds Python support for large, multi-dimensional arrays and matrices, along with a large library of high-level mathematical functions to operate on these arrays.
- `SciPy` A collection of mathematical algorithms and convenience functions built on the `Numpy` extension of Python. It adds significant power to the interactive Python session by providing the user with high-level commands and classes for manipulating and visualizing data.
- `Pandas` Software library written for data manipulation and analysis in Python. Offers data structures and operations for manipulating numerical tables and time series.
- `Scikit-learn` A Python module for machine learning built on top of `SciPy` and distributed under the 3-Clause BSD license.

## Table of Contents
##### 1. Preprocessing
- Imputation, Scaling
- Feature Extraction

##### 2. Unsupervised Learning
- Dimensionality Reduction
- Clustering

##### 3. Supervised Learning
- Regression
- Classification

##### 4. Model Selection
- Cross Validation
- Grid Search

##### 5. Pipelines

In this tutorial we will use the `Boston` dataset, and the `Iris` dataset, both included with `sklearn`

In [7]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston
boston = load_boston()

from sklearn.datasets import load_iris
iris = load_iris()

In [8]:
print(boston.DESCR)

In [9]:
print(iris.DESCR)

Convert the `.data` numpy arrays into pandas DataFrames for readability

In [11]:
boston_df = pd.DataFrame(boston.data, columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])
boston_df['MEDV'] = boston.target

iris_df = pd.DataFrame(iris.data, columns=['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width'])
iris_df['Species'] = iris.target

## 1. Preprocessing
http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing

The preprocessing _transformations_ of Scikit-learn are classes with two methods:
1. `fit` which takes one or more columns of a numpy array or pandas dataframe and records some information about the data in those columns
1. `transform` which takes one or more columns of an array or dataframe (possibly the same as above, possibly different) and then returns a __numpy array__ based on the the _fitted_ data and the columns to _transform_

#### Imputation
Imputation replaces missing values with values based on the valid values in the row or column.

In [16]:
from sklearn.preprocessing import Imputer

In [17]:
X = np.array([[1,     np.nan], 
             [np.nan, 4], 
             [7,      6]
            ])
X

We must initialize the imputer object with specifications before we can fit and transfrom `X`. We will name it `imp`.

In [19]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)

Above we have specified the imputer to replace `'NaN'` with the `mean` of each column (`axis=0`).

Below, note fitting does not change `X`, it instead calculates the mean of each column and stores them in our object, `imp`.

In [21]:
imp.fit(X)

In [22]:
imp.statistics_

The `transform` method creates, in this case, a new array with the missing values of `X` replaced with the mean of the column containing the missing value.

In [24]:
X_new = imp.transform(X)
X_new

Our data sets do not have any missing values. Below we remove some values from the first column of our iris dataset to illustrate imputation.

In [26]:
iris_with_missing = iris_df[:].copy()
iris_with_missing.iloc[[3, 4, 12, 34, 47, 54, 59, 63, 105, 115], 0] = np.nan
iris_with_missing.head()

In [27]:
iris_imp = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=1)
iris_imp.fit(iris_with_missing)

In [28]:
iris_imp

In [29]:
iris_with_missing

In [30]:
pd.DataFrame(iris_filled, columns = iris_df.columns)

Note that our dataframe is now a numpy array - we have lost our column headings and indices

__Exercise:__ Try changing the parameters in the examples above:
- strategy = `mean`, `median`, `most_frequent`
- axis = `0` (replace based on column), `1` (replace based on row)

In [33]:
iris_imp = Imputer(missing_values='NaN', strategy='median', axis=0)
iris_imp.fit(iris_with_missing)
iris_imp.transform(iris_with_missing)

#### Scaling
Some machine learning methods assume that a column of data is in a specific range (for instance, 0 to 1) or that the data has a mean of 0 and a standard deviation of 1. Scalers transform the values of a dataframe to meet these assumptions. They are especially useful before applying distance-based models, where a variable with unusually large values can outweight the rest. 

Options:
- `MinMaxScaler` - Scales column data such that the minimum for each column is 0 and the maximum is 1
- `MaxAbsScaler` - Scales column data such that the maximum absolute value of each column is 1
- `StandardScaler` - Standardizes column data such that the mean of each column is 0 and the standard deviation of each is 1
- `Normalizer` - Scales __row__ data such that each row is a vector of length 1

In [35]:
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler, Normalizer

__Exercise__: Compare each of the scalers below on `X` as well as our `iris_df`

In [37]:
X = np.array([[ 1., -1.,  2.],
              [ 100.,  0.,  0.],
              [ 0.,  1., -1.]])
X

In [38]:
scaler = MinMaxScaler()

scaler.fit(X)
scaler.transform(X)

In [39]:
iris_df.head()

In [40]:
iris_measurements = iris_df.drop('Species', axis=1)
iris_measurements.describe()

In [41]:
scaler_iris = MinMaxScaler()

scaler_iris.fit(iris_measurements)
iris_scaled = scaler_iris.transform(iris_measurements)
pd.DataFrame(iris_scaled, columns=['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width']).describe()

In [42]:
pd.DataFrame(iris_scaled, columns=['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width']).head()

#### Feature Extraction
Feature extraction transforms variables so that they are more friendly to machine learning. Some examples include:
- Converting continuous variables to binary
- Converting categorical variables to binary dummy variables
- Converting raw sentences to word counts

In [45]:
from sklearn.preprocessing import Binarizer, LabelBinarizer, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

With `Binarizer` values above a given threshold become 1, and values equal-to or below the threshold become 0. We will demonstrate on our original array, `X`.

In [47]:
X

Below, instead of applying both `.fit()` and `.transform()`, we can instead equivalently use `.fit_transform()`

In [49]:
binarizer = Binarizer(threshold=0)

print(binarizer.fit(X).transform(X))
print('')
print(binarizer.fit_transform(X))

`LabelBinarizer`
- Takes as input (to the `fit` method) a multi-class 1-dimensional feature (list of __strings__)
- Return as output (by the `transform` method) one or more binary features (numpy array)

`OneHotEncoder`
- Takes as input (to the `fit` method) a multi-class 1-dimensional feature (of categorical __integers__)
- Return as output (by the `transform` method) one or more binary features (numpy array)

In [51]:
lb = LabelBinarizer()
lb.fit(['A', 'B', 'A', 'C'])

In [52]:
lb.transform(['A', 'B', 'A', 'C'])

Above, `.fit()` determines that there are 3 unique values to be turned into binary columns.

Below, notice "D" is lost from our new vector because it was not part of our fit procedure.

In [54]:
lb.transform(['A', 'C', 'F'])

__Exercise:__ Create your own 1D array and apply `LabelBinarizer`.

In our `iris` dataset, our target variable `Species` is encoded as follows:
- Iris-Setosa (0)
- Iris-Versicolour (1)
- Iris-Virginica (2)

Using `OneHotEncoder`, we can convert this into three binary variables.

In [58]:
iris_df['Species'].value_counts()

In [59]:
ohe = OneHotEncoder()
targets = ohe.fit_transform(iris_df['Species'].reshape(-1, 1)) # .reshape(-1, 1) removes index from iris_df['Species]
targets.toarray() # Converts sparse matrix to array

`CountVectorizer`
- Takes as input (to the `fit` method) an array or list of __strings__
- Return as output (by the `transform` method) a sparse matrix of token counts, in alphabetical order of tokens. With default settings tokens are words, and the vectorizer removes punctuation and lowers the case.

In [61]:
documents = ['This is the first document.',
             'This is the second second document.',
             'And the third one.',
            ]

In [62]:
vectorizer = CountVectorizer()
vectorizer.fit(documents)
print(vectorizer.get_feature_names()) # Columns

matrix = vectorizer.transform(documents)
print(matrix.toarray()) # Counts

In [63]:
pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())

__Exercise:__ Write and vectorize your own sentences.

## 2. Unsupervised Learning

#### Dimensionality Reduction
The most common form of dimensionality reduction is `Principal Components Analysis`

Wikipedia: https://en.wikipedia.org/wiki/Principal_component_analysis
> Principal component analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components.

In [68]:
from sklearn.decomposition import PCA

If we return to our `iris` dataset, some of our predictor variables are highly correlated. PCA can help with this.

In [70]:
iris_measurements.corr()

Below, `.fit_transform()` returns a numpy array, which we convert again to a pandas DataFrame for readability.

In [72]:
iris_measurements.head()

In [73]:
pca = PCA()
iris_pca = pca.fit_transform(iris_measurements) 
iris_pca = pd.DataFrame(iris_pca, columns=['PC1', 'PC2', 'PC3', 'PC4'])
iris_pca.head()

Our 4 variables are now measured in terms of our 4 principal components. By default, we always return as many principal components as there are variables. Note our 4 principal components are completely uncorrelated.

In [75]:
np.round(iris_pca.corr(),2)

The other benefit of `PCA` is that the principal components are sorted in decreasing order of importance. Below, we can see that our first principal component explains __92%__ of the variation in our data, our second component explains __5%__, and the other two explain even less.

In [77]:
pca.explained_variance_ratio_

The __Reduction__ part of Dimensionality Reduction comes from the fact that we can limit the number of principal components we want to keep in the returned dataset. We'll initialize the object `pca` to just use the first two, which explain almost 98% of the variation.

In [79]:
pca = PCA(n_components = 2)
iris_pca = pca.fit_transform(iris_measurements) 
iris_pca = pd.DataFrame(iris_pca, columns=['PC1', 'PC2'])
iris_pca.head()

__Exercise:__ Use PCA on the `boston_X` dataset below _(target variable `MEDV` removed)_. How many principal components would you keep?

In [81]:
boston_X = boston_df.drop('MEDV', axis=1)
boston_X.corr()

In [82]:
pca = PCA()
pca.fit(boston_X)
pca.explained_variance_ratio_.round(2)

In [83]:
pd.DataFrame(pca.components_[0].round(2).reshape(-1,13), columns=boston_X.columns)

In [84]:
pca = PCA(n_components = 2)
pca.fit_transform(boston_X)

#### Clustering
Clustering groups similar data points into sets. There are many variants. We will focus on `K-Means`.

Wikipedia: https://en.wikipedia.org/wiki/K-means_clustering
> K-means clustering aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean, serving as a prototype of the cluster

In [86]:
from sklearn.cluster import KMeans

We will use our `iris_scaled` array from earlier. Because `KMeans` is distance based, it would be biased to separate clusters along features with different scales.

From the documentation, we know these plants belong to three species. Lets try to create three clusters and see how they compare to the real groupings. As a reminder, the first 5 rows of our scaled array are printed below.

In [88]:
iris_scaled[0:5]

Below, we first specify a `random_state`. There is some randomness involved in identifying clusters, so this enables reproducibility. The `.fit()` method divides the data into clusters, and calculates the centers of each cluster. The `.predict()` method then labels data points according to the cluster centers calculated earlier.

In [90]:
km = KMeans(n_clusters=3, random_state=1)
km.fit(iris_scaled)
clusters = km.predict(iris_scaled)

In [91]:
pd.crosstab(clusters, iris_df['Species'])

Not too bad! The cluster numbers are meaningless, but we can see that species 0 (column) is completely identified in a cluster, species 1 (column) is almost completely identified in a cluster, and species 2 is split between two clusters.

## 3. Supervised Learning

The supervised learning _estimators_ of Scikit-learn are classes with two methods:
1. `fit` which takes a numpy array or pandas dataframe of predictor variables as well as a 1D array or dataframe for the target variable
1. `predict` which takes a numpy array or pandas dataframe with the same columns as the predictor variables and returns a __numpy array__ of estimates based on the the _fitted_ data

_Important note: all data must be numeric (or transformed to numeric from categories or labels above)_

#### Regression
Predicting a continuous-valued attribute. We will use our `Boston` dataset, to predict `MEDV` (Median Home Value) based on the other predictors. Let's first split our data into training and test sets, so we can evaluate how well it predicts for unseen data.

_Note: `train_test_split` converts our data into numpy arrays_

In [97]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(boston_df.drop('MEDV', axis=1),
                                                    boston_df['MEDV'],
                                                    test_size = 0.33, random_state=1
                                                   )

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In this example we will use a simple `LinearRegression` model, and look at `Mean Squared Error` and `R-squared` to assess our predictions. We `.fit()` our model to the training data, `.predict()` based on the test data, and score the true y-values against our predictions.

In [99]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lm = LinearRegression()
lm.fit(X_train, y_train)

y_pred = lm.predict(X_test)

print('MSE:', mean_squared_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))

__Exercise:__ Use a linear regression to predict `CRIM` (crime rate) based on all other predictors. Make sure to split the data into training and test sets.

In [101]:
boston_df.head()

#### Classification
Identifying to which category an object belongs to. We will use our __scaled__ `Iris` dataset to predict species using a `LogisticRegression` and evaluate the `accuracy score` (percent correct classes). Again we will split our data into training and test sets, `.fit()` to the training set, `.predict()` based on the test set, and score those predictions against the true categories.

In [105]:
X_train, X_test, y_train, y_test = train_test_split(iris_scaled,
                                                    iris_df['Species'],
                                                    test_size = 0.33, random_state=1
                                                   )

In [106]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logfit = LogisticRegression(multi_class='multinomial', solver='newton-cg')
logfit.fit(X_train, y_train)

y_pred = logfit.predict(X_test)
print(pd.crosstab(y_pred, y_test))

print('')
print('accuracy:', accuracy_score(y_test, y_pred))

We can also extract the predicted probabilities for each class. The prediction above is based on the class with the highest probability. 

Below, we print the first 5 rows of these predicted probabilities.

In [108]:
y_probs = logfit.predict_proba(X_test)
y_probs[0:5]

For a complete list of classification and regression models, see http://scikit-learn.org/stable/supervised_learning.html#supervised-learning

## 4. Model Selection

To avoid overfitting, earlier we split our data into training and test sets, and only scored the model based on the test set. While this is a good process for understanding our final model's predictive power, if we leverage the test data too early in our process we risk overfitting by model choice. 

For this reason, the best process is to do all of our fitting, model selection, and parameter tuning on _only the training data_, and to not use the test data until we have chosen our model and parameters. Only then does our test data gives us a true score.

#### Cross Validation
Cross validation is the process of dividing data into equal subsets, and using each subset for scoring a model fit on all other subsets.
Wikipedia: https://en.wikipedia.org/wiki/Cross-validation_(statistics)

![alt](https://upload.wikimedia.org/wikipedia/commons/1/1c/K-fold_cross_validation_EN.jpg)

In scikit-learn, `cross_val_score` implements this process for a variety of metrics 
- Documentation: http://scikit-learn.org/stable/modules/cross_validation.html
- List of metrics: http://scikit-learn.org/stable/modules/model_evaluation.html

Below, we will use cross validation on our training data from the Iris classification problem earlier. By setting `cv=5`, we divide our data into 5 folds.

In [117]:
from sklearn.model_selection import cross_val_score

logfit = LogisticRegression(multi_class='multinomial', solver='newton-cg')
scores = cross_val_score(logfit, 
                         X_train, 
                         y_train, 
                         scoring='accuracy', 
                         cv=5)

print(scores)
print(scores.mean())

__Exercise:__ Pick a different metric and score the training data.

#### Grid Search
If our model has parameters we can tune, `GridSearchCV` can help. It implements the cross-validation process above, but calculates scores for a grid of possible input parameters. We can then use the best scoring model to make predictions.

In [121]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

For this example, we will use `KNeighborsClassifier`, which is a supervised learning method that labels data points according to the most common class among its  `K` closest data points. For small values of `K`, the model draws a jagged boundary that is more susceptible to noise but can also fit nonlinear patterns. For larger values of `K`, the boundary between classes is smoothed. We will use `GridSearchCV` to find the ideal value of `K`.

In [123]:
parameters = {'n_neighbors': [1, 5, 7, 11, 13],
              'weights': ['uniform', 'distance']
             }

knn = KNeighborsClassifier()
grid = GridSearchCV(estimator=knn,
                    param_grid = parameters,
                    scoring = 'accuracy',
                    cv = 3
                   )

Notes on our parameters above:
- For `n_neighbors`, because we have 3 classes and want to avoid ties, we do not use even numbers or numbers divisible by 3
- For weights, `'uniform'` gives all points equal weight, `'distance'` gives more weight to closer points

Below, we fit our grid object to the scaled training data.

In [125]:
grid.fit(X_train, y_train)

print('Best Accuracy Score:', grid.best_score_)
print('Best Parameters:', grid.best_params_)

The best-scoring model fits to the __5__ nearest neighbors, and gives each neighbor an equal weight. We can also make predictions using the grid object - it uses the best model. This was our best model and best parameters above, we could now bring back the test data.

In [127]:
y_pred = grid.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))

__Exercise:__ Make some changes to the parameter grid, and change the scoring metric. Are there any changes to the best model?

## 5. Pipelines

There are a lot of steps above to transform data in the way we want. Luckily, we can combine all of our transformers and an estimator into a pipeline object to speed up the process. Let's return to our messy iris data from earlier (`iris_with_missing`), and split that into training and test data.

In [133]:
from sklearn.pipeline import Pipeline

In [134]:
iris_with_missing.head()

In [135]:
X_train, X_test, y_train, y_test = train_test_split(iris_with_missing.drop('Species', axis=1),
                                                    iris_with_missing['Species'],
                                                    test_size = 0.33, random_state=1
                                                   )

A `Pipeline` object takes as input a list of tuples where each tuple has: 
- A name (`string`)
- A transformer object 

The objects run in sequence, and the last item in a `Pipeline` is typically an estimator object. Below we use an `Imputer`, `MinMaxScaler`, `PCA`, and our `KNeighborsClassifier` estimator.

In [137]:
Pipe = Pipeline([('imputer', Imputer(strategy='mean')),
                 ('scaler' , MinMaxScaler()),
                 ('pca'    , PCA()),
                 ('knn'    , KNeighborsClassifier(weights='uniform'))
                 ])

Notice we haven't done anything with our data yet. We will now fit the pipeline to the training data, use `GridSearchCV` to select the best parameters, and finally make predictions on the test data. The pipeline will transform the test data exactly in the way it transforms the training data.

To use a grid search with a pipeline, we must identify the step name from above, followed by two underscores, followed by the parameter name

In [140]:
parameters = {'pca__n_components': [1, 2, 3, 4],
              'knn__n_neighbors': [5, 7, 11]
             }

grid = GridSearchCV(estimator=Pipe,
                    param_grid = parameters,
                    scoring = 'accuracy',
                    cv = 3
                   )

In [141]:
grid.fit(X_train, y_train)

print('Best Accuracy Score:', grid.best_score_)
print('Best Parameters:', grid.best_params_)

In [142]:
y_pred = grid.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))

__Exercise:__ Create and fit a pipeline to the `Boston` dataset. The pipeline should include a `scaler`, `PCA`, and `LinearRegression`. Use `GridSearchCV` to determine the best number of PCA components to include (based on `Mean Squared Error`).

What is the `Mean Squared Error` of final predictions on the test data?

In [144]:
boston_df.head()

__The End__